In [49]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
import re

In [75]:
#add a year from pro football reference
def cleanFP(dfwr):
    dfwr['Player'] = dfwr['Player'].str.split("(").str[0]
    dfwr['YDS'] = dfwr['YDS'].str.replace(',', '').astype(float)
    #dfwr['Fpts/G'] = dfwr.apply(lambda row: (row.REC + row.YDS / 10 + row.TD * 6) / row.G, axis=1)
    dfwr = dfwr.drop(columns=['AIR', 'AIR/R'])
    return dfwr

dfwr22 = pd.read_csv(r"C:\Users\luker\Downloads\Python\WR22.csv")
dfwr22 = cleanFP(dfwr22)
dfwr22.dtypes

Rank         float64
Player        object
G            float64
REC          float64
YDS          float64
Y/R          float64
YBC           object
YBC/R        float64
YAC          float64
YAC/R        float64
YACON        float64
YACON/R      float64
BRKTKL       float64
TGT          float64
% TM          object
CATCHABLE    float64
DROP         float64
RZ TGT       float64
10+ YDS      float64
20+ YDS      float64
30+ YDS      float64
40+ YDS      float64
50+ YDS      float64
LNG          float64
dtype: object